In [17]:
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import coo_array
from pathlib import Path
import networkx as nx
import pandas as pd
import numpy as np
import itertools
# from cdlib import algorithms

# 1. Zhang et al. Method
This method uses the pagerank technique outlined in [Zhang *et al.,* 2022](https://arxiv.org/abs/2104.02764). 

Mainly: 

Let $M$ be a transition matrix with: 
$$m_{ij} = \begin{cases}
    \theta w_{ji}/s_j^{out} + (1-\theta)a_{ji}/d_j^{out} && \texttt{if}\; d_j^{out}\neq 0\\
    0 && \texttt{if}\; d_j^{out}=0
\end{cases}$$
Where: 
- $\theta$ is a tunable parameter that represents how important edge weights should be in the pagerank alg
- $w_{ji}$ is the weight of an edge from node $j \rightarrow i$
- $s_j^{out} = \sum_{v \in V|j \rightarrow v}w_{jv}$ is the "strength" of outgoing edges from node $j$
- $a_{ji}$ = $1 \:\texttt{if}\: j \rightarrow i \:\texttt{else}\: 0$
- $d_j^{out} = \sum_{v \in V|j \rightarrow v}a_{jv}$
- $\beta_i$ is the node importance score
  


Then the pagerank can be calculated with power iterations on
$$P=\gamma MP + (1-\gamma)\boldsymbol{\beta}/||\boldsymbol{\beta}||_1$$

Where:
- $1-\gamma$ is a tunable parameter representing the probability of restarting a random walk (typically 0.8-0.9)
- $\boldsymbol{\beta}/||\boldsymbol{\beta}||_1$ is the vectorized version of $\beta_{i}/\sum_{i\in V}\beta_i$

---
*note the other formulation in the paper results in a dense transition matrix that is nxn.... too big*

*note that for $m_{ij}$, $i$ is the target and $j$ is the source*

In [18]:
THETA = 0.8
GAMMA = 0.9
REVERSE = True

DATAPATH = Path('../data/processed/pagerank')
OUTPATH = DATAPATH / 'output.parquet'

n_df = pd.read_parquet(DATAPATH / 'input_node_scores.parquet')
e_df = pd.read_parquet(DATAPATH / 'input_edge_scores.parquet')
e_df = e_df[['cust_id_sender', 'cust_id_receiver', 'score']].copy()

In [19]:
#reversing edges
if REVERSE: 
    e2_df = pd.DataFrame()
    e2_df['cust_id_receiver'] = e_df['cust_id_sender'].copy()
    e2_df['cust_id_sender'] = e_df['cust_id_receiver'].copy()
    e2_df['score'] = e_df['score']

    e_df = pd.concat([e_df, e2_df])
    e_df.sample(3)

## Constructing $\boldsymbol{\beta}/||\boldsymbol{\beta}||_1$

In [20]:
# Get all nodes in the graph
node_list = []
node_list.extend(e_df.cust_id_sender.tolist() + e_df.cust_id_receiver.tolist() + n_df.cust_id.tolist())
node_list = list(set(node_list)) #hack to remove duplicate"

In [21]:
# Construct B
b_df = pd.DataFrame(data={'cust_id':node_list})
b_df = b_df.merge(n_df, on='cust_id', how='left')
b_df = b_df.rename(columns={'score':'b_i'})
b_df = b_df.fillna(0)

one_norm = abs(b_df['b_i']).sum()

b_df['b_i'] = b_df['b_i'] / one_norm

b_df.sort_values('b_i', ascending=False).head(3)

,cust_id,b_i
50110,CUST23925850,0.000083
174387,CUST84626197,0.000083
141206,CUST29413585,0.000083


## Node Encoding
This section encodes the cust_ids ordinally.

In [22]:
le = LabelEncoder()
le.fit(node_list)

e_df['cust_id_sender'] = le.transform(e_df['cust_id_sender'])
e_df['cust_id_receiver'] = le.transform(e_df['cust_id_receiver'])

b_df['cust_id'] = le.transform(b_df['cust_id'])

node_enc = le.transform(node_list)

e_df.sample(3)

,cust_id_sender,cust_id_receiver,score
494546,50555,240147,0.0
170120,50386,228220,0.0
49088,214440,101727,0.0


## Constructing $M$
*note that for $m_{ij}$, $i$ is the target node and $j$ is the source node*

In [23]:
e_df['score'] += 0.01 #No edges can have 0 weight, so we add a relatively small value to all edges

#Calculate w_ji
w_df = e_df.groupby(['cust_id_sender', 'cust_id_receiver'], as_index=False)['score'].sum()
w_df = w_df.rename(columns={'score':'w_ji'})
e_df = e_df.merge(w_df, on=['cust_id_sender', 'cust_id_receiver'], how='left')

#Calculate a_ji
a_df = e_df.groupby(['cust_id_sender', 'cust_id_receiver'], as_index=False)['score'].count()
# a_df = e_df.groupby 
a_df = a_df.rename(columns={'score':'a_ji'})
e_df = e_df.merge(a_df, on=['cust_id_sender', 'cust_id_receiver'], how='left')

#Calculate s_j
s_df = e_df.groupby(['cust_id_sender'], as_index=False)['score'].sum()
s_df = s_df.rename(columns={'score':'s_j'})
e_df = e_df.merge(s_df, on='cust_id_sender', how='left')

#Calculate d_j
d_df = e_df.groupby(['cust_id_sender'], as_index=False)['score'].count()
d_df = d_df.rename(columns={'score':'d_j'})
e_df = e_df.merge(d_df, on='cust_id_sender', how='left')

#Remove duplicate edges... taking max is just a hack
e_df = e_df.groupby(['cust_id_sender', 'cust_id_receiver'], as_index=False).max()

e_df.sample(10)

,cust_id_sender,cust_id_receiver,score,w_ji,a_ji,s_j,d_j
694262,172069,97432,0.010000,0.010000,1,0.020000,2
73742,18061,214915,0.010000,0.010000,1,0.010000,1
539706,133144,72333,0.510000,0.510000,1,8.650000,15
988835,254013,192956,0.010000,0.010000,1,0.870000,12
480538,118520,197099,0.843333,0.843333,1,7.973333,14
353513,87217,266010,0.010000,0.010000,1,0.050000,5
139875,34104,260381,0.010000,0.010000,1,0.010000,1
300629,73921,285072,0.510000,0.510000,1,2.550000,5
86789,21239,43342,0.010000,0.010000,1,1.160000,16
961644,246087,25918,0.510000,0.510000,1,2.860000,11


In [24]:
def calc_m(r): 
    m = THETA*r.w_ji/r.s_j + (1-THETA)*r.a_ji/r.d_j
    return m

#calculate m_ij for sender j, receiver i
e_df['m'] = e_df.apply(lambda r: calc_m(r), axis=1)

In [25]:
#Verify matrix is column stochastic... output m should be 1
e_df.groupby('cust_id_sender').sum().sort_values('m', ascending=True).head(3)

,cust_id_receiver,score,w_ji,a_ji,s_j,d_j,m
cust_id_sender,,,,,,,
38929,3977510,0.73,0.73,23,16.79,529,1.0
64911,4000942,0.73,0.73,23,16.79,529,1.0
142376,876084,2.07,2.07,7,14.49,49,1.0


## Pagerank

In [26]:
def weighted_pagerank(M, B, P, gamma=0.85, tol=1e-10, maxit=1000):
    """Computes the node and edge weighted pagerank
    
    This function takes in a transition matrix, node weights, and an initial pagerank vector
    and computes the weighted pagerank, stopping when the difference between iterations is < 
    tolerence or when the maximum # iterations is reached.
    
    Args:
        M: The transition matrix according to Zhang et al. should be a scipy sparse coordinate matrix (NxN)
        B: A numpy array of relative node weights (Nx1)
        P: A numpy array vector of initial pageranks, normally just uniform (Nx1) 
        gamma: A hyperparam representing the probability of a random surfer NOT making a random jump to a new node
        tol: iteration tolerance - when the maximum RELATIVE change in a node score is below this value, iterations stop
        maxit: maximum iterations
    
    Returns
        P: The final pagerank vector, min-max scaled to be in [0-1]
    
    """
    
    itcount = 0
    max_diff = 1000 #placeholder large value
    diff_list = []
    while (itcount <= maxit) and (max_diff >= tol):
           
        #Pagerank iter
        P_int = GAMMA*M.dot(P) + (1-GAMMA)*B
                
        #Adding leaked pagerank back
        #need to do this since we don't preprocess to remove dead ends.
        leak = np.sum(P_int)
        P_int = P_int + (1-leak)/len(P)
        
        max_diff = (np.absolute(P-P_int)/P).max()
        diff_list.append(max_diff)
        itcount += 1
        P = P_int
        
    
    print(f'Final max (relative) error: {max_diff:.3}')
    print(f'Final iterations: {itcount}')
    # Scaling
    P = (P - P.min())/(P.max() - P.min())
    
    return P, diff_list

In [27]:
#Construct necessary matrices
i = e_df['cust_id_receiver'].values
j = e_df['cust_id_sender'].values
m = e_df['m'].values

N = b_df.shape[0]
B = b_df.sort_values('cust_id')['b_i'].values
M = coo_array((m,(i,j)), shape=(N,N))
P = np.full(N, 1/N)

#Run Pagerank
P, dl = weighted_pagerank(M, B, P, GAMMA)

Final max (relative) error: 8.98e-11
Final iterations: 245


In [28]:
# Exporting
P_df = pd.DataFrame(data=P, columns=['score'])
P_df['cust_id'] = P_df.index
P_df['cust_id'] = le.inverse_transform(P_df['cust_id']) #transform back to actual IDS
P_df = P_df[['cust_id','score']]
P_df.to_parquet(OUTPATH)

# 2. Custom Method

This method adapts the pagerank technique outlined in [Zhang *et al.,* 2022](https://arxiv.org/abs/2104.02764). In that work, the random walk uses relative edge weights to assign probabilities of traversing edges from a given node. This means edges are weighted *locally* - around a given node - but not *globally*. 

In this method, I use the weighted pagerank method from Zhang *et al.,*, but I adjust the gamma parameter - the probability of randomly teleporting from any node - to be *node specific* and based on the total edge suspicion of a given node.

Mainly: 

Let $M$ be a transition matrix with: 
$$m_{ij} = \begin{cases}
    w_{ji}/s_j^{out} && \texttt{if}\; d_j^{out}\neq 0\\
    0 && \texttt{if}\; d_j^{out}=0
\end{cases}$$
Where: 
- $\theta$ is a tunable parameter that represents how important edge weights should be in the pagerank alg
- $w_{ji}$ is the weight of an edge from node $j \rightarrow i$
- $s_j^{out} = \sum_{v \in V|j \rightarrow v}w_{jv}$ is the "strength" of outgoing edges from node $j$
- $a_{ji}$ = $1 \:\texttt{if}\: j \rightarrow i \:\texttt{else}\: 0$
- $d_j^{out} = \sum_{v \in V|j \rightarrow v}a_{jv}$
- $\beta_i$ is the node importance score
  


Then the pagerank can be calculated with power iterations on
$$P=\boldsymbol{\gamma} MP + (1-\boldsymbol{\gamma})\boldsymbol{\beta}/||\boldsymbol{\beta}||_1$$

Where:
- $1-\boldsymbol{\gamma}$ is a vector of hyperparameters representing the probability of randomly teleporting from a given node
- $\boldsymbol{\beta}/||\boldsymbol{\beta}||_1$ is the vectorized version of $\beta_{i}/\sum_{i\in V}\beta_i$

---
*note the other formulation in the paper results in a dense transition matrix that is nxn.... too big*

*note that for $m_{ij}$, $i$ is the target and $j$ is the source*

In [121]:
DATAPATH = Path('../data/processed')
REVERSE = True

n_df = pd.read_parquet(DATAPATH / 'pr_node_score.parquet')
e_df = pd.read_parquet(DATAPATH / 'pr_edge_score.parquet')
e_df = e_df[['cust_id_sender', 'cust_id_receiver', 'score']].copy()

In [ ]:
#reversing edges
if REVERSE:
    e2_df = pd.DataFrame()
    e2_df['cust_id_receiver'] = e_df['cust_id_sender'].copy()
    e2_df['cust_id_sender'] = e_df['cust_id_receiver'].copy()
    e2_df['score'] = e_df['score']

    e_df = pd.concat([e_df, e2_df])
    e_df.sample(3)

## Constructing $\boldsymbol{\beta}/||\boldsymbol{\beta}||_1$

In [ ]:
# Get all nodes in the graph
node_list = []
node_list.extend(e_df.cust_id_sender.tolist() + e_df.cust_id_receiver.tolist() + n_df.cust_id.tolist())
node_list = list(set(node_list)) #hack to remove duplicate"

In [ ]:
# Construct B
b_df = pd.DataFrame(data={'cust_id':node_list})
b_df = b_df.merge(n_df, on='cust_id', how='left')
b_df = b_df.rename(columns={'score':'b_i'})
b_df = b_df.fillna(0)

one_norm = abs(b_df['b_i']).sum()

b_df['b_i'] = b_df['b_i'] / one_norm

b_df.sort_values('b_i', ascending=False).head(3)

## Node Encoding
This section encodes the cust_ids ordinally.

In [ ]:
le = LabelEncoder()
le.fit(node_list)

e_df['cust_id_sender'] = le.transform(e_df['cust_id_sender'])
e_df['cust_id_receiver'] = le.transform(e_df['cust_id_receiver'])

b_df['cust_id'] = le.transform(b_df['cust_id'])

node_enc = le.transform(node_list)

e_df.sample(3)

## Constructing $M$
*note that for $m_{ij}$, $i$ is the target node and $j$ is the source node*

In [ ]:
e_df['score'] += 0.01 #No edges can have 0 weight, so we add a relatively small value to all edges

#Calculate w_ji
w_df = e_df.groupby(['cust_id_sender', 'cust_id_receiver'], as_index=False)['score'].sum()
w_df = w_df.rename(columns={'score':'w_ji'})
e_df = e_df.merge(w_df, on=['cust_id_sender', 'cust_id_receiver'], how='left')

#Calculate a_ji
a_df = e_df.groupby(['cust_id_sender', 'cust_id_receiver'], as_index=False)['score'].count()
# a_df = e_df.groupby 
a_df = a_df.rename(columns={'score':'a_ji'})
e_df = e_df.merge(a_df, on=['cust_id_sender', 'cust_id_receiver'], how='left')

#Calculate s_j
s_df = e_df.groupby(['cust_id_sender'], as_index=False)['score'].sum()
s_df = s_df.rename(columns={'score':'s_j'})
e_df = e_df.merge(s_df, on='cust_id_sender', how='left')

#Calculate d_j
d_df = e_df.groupby(['cust_id_sender'], as_index=False)['score'].count()
d_df = d_df.rename(columns={'score':'d_j'})
e_df = e_df.merge(d_df, on='cust_id_sender', how='left')

#Remove duplicate edges... taking max is just a hack
e_df = e_df.groupby(['cust_id_sender', 'cust_id_receiver'], as_index=False).max()

e_df.sample(10)

In [ ]:
def calc_m(r): 
    m = THETA*r.w_ji/r.s_j + (1-THETA)*r.a_ji/r.d_j
    return m

#calculate m_ij for sender j, receiver i
e_df['m'] = e_df.apply(lambda r: calc_m(r), axis=1)

In [ ]:
#Verify matrix is column stochastic... m should all be 1.0
e_df.groupby('cust_id_sender').sum().sort_values('m', ascending=True).head(3)

## Constructing $\boldsymbol{\gamma}$

## Pagerank

In [ ]:
def custom_pagerank(M, B, P, gamma, tol=1e-10, maxit=1000):
    """Computes the node and edge weighted pagerank using a custom algorithm
    
    This function takes in a transition matrix, node weights, and an initial pagerank vector
    and computes the weighted pagerank, stopping when the difference between iterations is < 
    tolerence or when the maximum # iterations is reached.
    
    Args:
        M: The transition matrix according to Zhang et al. should be a scipy sparse coordinate matrix (NxN)
        B: A numpy array of relative node weights (Nx1)
        P: A numpy array vector of initial pageranks, normally just uniform (Nx1) 
        gamma: An array of probabilities of randomly teleporting from a given node (Nx1)
        tol: iteration tolerance - when the maximum RELATIVE change in a node score is below this value, iterations stop
        maxit: maximum iterations
    
    Returns
        P: The final pagerank vector, min-max scaled to be in [0-1]
    
    """
    
    itcount = 0
    max_diff = 1000 #placeholder large value
    diff_list = []
    while (itcount <= maxit) and (max_diff >= tol):
           
        #Pagerank iter
        P_int = GAMMA*M.dot(P) + (1-GAMMA)*B
                
        #Adding leaked pagerank back
        #need to do this since we don't preprocess to remove dead ends.
        leak = np.sum(P_int)
        P_int = P_int + (1-leak)/len(P)
        
        max_diff = (np.absolute(P-P_int)/P).max()
        diff_list.append(max_diff)
        itcount += 1
        P = P_int
        
    
    print(f'Final max (relative) error: {max_diff:.3}')
    print(f'Final iterations: {itcount}')
    # Scaling
    P = (P - P.min())/(P.max() - P.min())
    
    return P, diff_list

In [ ]:
#Construct necessary matrices
i = e_df['cust_id_receiver'].values
j = e_df['cust_id_sender'].values
m = e_df['m'].values

N = b_df.shape[0]
B = b_df.sort_values('cust_id')['b_i'].values
M = coo_array((m,(i,j)), shape=(N,N))
P = np.full(N, 1/N)

#Run Pagerank
P, dl = weighted_pagerank(M, B, P, GAMMA)

In [ ]:
# Exporting
P_df = pd.DataFrame(data=P, columns=['score'])
P_df['cust_id'] = P_df.index
P_df['cust_id'] = le.inverse_transform(P_df['cust_id']) #transform back to actual IDS
P_df = P_df[['cust_id','score']]
P_df.to_parquet(DATAPATH / 'pagerank_rev_edge.parquet')

# Vis

In [66]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.io as pio
import kaleido 
print('kaleido version:', kaleido.__version__)

#Inline figures
pio.renderers.default = 'iframe'

#Themeing
# mcolors = px.colors.qualitative.Dark24

pio.templates['custom'] = go.layout.Template(
    layout=dict(
        xaxis=dict(ticks='outside', tickcolor='lightgray', showgrid=False, showline=True),
        yaxis=dict(ticks='outside', tickcolor='lightgray', showgrid=False, showline=True, mirror=True),
        yaxis2=dict(ticks='outside', tickcolor='lightgray', showgrid=False,),
        # colorway=mcolors,
    )

)

pio.templates.default = 'plotly_white+custom'

kaleido version: 0.2.1
